## Lesson 1 - 01. pytorch-basics

In [40]:
import torch

In [48]:
# How do you create a PyTorch tensor? Illustrate with examples.

# one-dimensional tensor (number)
t1 = torch.tensor(1.)
print(t1)
print(t1.dtype)

# two-dimensional tensor (matrix)
t2 = torch.tensor([1., 2])
print(t2)
print(t2.dtype)

# three-dimentional tensor
t3 = torch.tensor([[1., 2], [3, 4]])
print(t3)

tensor(1.)
torch.float32
tensor([1., 2.])
torch.float32
tensor([[1., 2.],
        [3., 4.]])


In [53]:
# How do you inspect the number of dimensions of a tensor and the length along each dimension?

# from outer most bracket to inner brackets, one only needs to count the number of elements

t4 = torch.tensor([[[1., 2, 3], [2, 3, 4]], [[4, 5, 6], [6, 7, 8]]])
print(t4)

# the .shape attributes will give the size: 2, 2, 3
print(t4.shape)

tensor([[[1., 2., 3.],
         [2., 3., 4.]],

        [[4., 5., 6.],
         [6., 7., 8.]]])
torch.Size([2, 2, 3])


In [64]:
# What happens if you specify requires_grad=True while creating a tensor? Illustrate with an example.
# What is autograd in PyTorch? How is it useful?
# What happens when you invoke the backward method of a tensor?
# How do you check the derivates of a result tensor w.r.t. the tensors used to compute its value?

# if 'requires_grad=True' is set for x, then we can use 'backward' to get a derivative of a variable defined with x w.r.t. x
# what the 'backward' does is called: 'autograd'

x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)

y = w * x + b
y.backward()
# z = 2 * w * x + b
# z.backward()
# the .grad accumulates the values; e.g., w.grad = 3 + 6 = 9

print(x.grad) # x does not have 'requires_grad=True'
print(w.grad) # dy/dw
print(b.grad) # dy/db

None
tensor(3.)
tensor(1.)


In [68]:
# How do you create a Numpy array?
# How do you create a PyTorch tensor using a Numpy array?
# How do you create a Numpy array using a PyTorch tensor?
# Why is interoperability between PyTorch and Numpy important?
# What is the purpose of a library like PyTorch if Numpy already provides data structures and utilities to with multi-dimensional numeric data?

x = np.array([[1, 2], [3, 4.]])
y = torch.from_numpy(x)
z = y.numpy()

# the interoperability is necessary because most datasets will be read and preprocessed as Numpy arrays.
# autograd : automaticaaly compute gradients, necessary for training deep learning models (SGD)
# GPU support : efficient for massive datasets and large models.

array([[1., 2.],
       [3., 4.]])

## Lesson 1 - 02. linear-regression

yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1

yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

In [69]:
import numpy as np
import torch

In [77]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [78]:
# Linear regression model from scratch

# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

preds = inputs @ w.t() + b
print(preds)
print(targets)

tensor([[-1.8432,  0.7313,  0.3656],
        [ 0.3596,  1.4202,  0.5982]], requires_grad=True)
tensor([ 1.4709, -0.5482], requires_grad=True)
tensor([[ -68.3643,  146.5749],
        [ -78.5068,  195.4325],
        [ -39.6869,  255.7326],
        [-141.5627,  119.3308],
        [ -29.9120,  202.4713]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [91]:
# prediction function
def model(x):
    return x @ w.t() + b

In [92]:
# loss function (MSE)
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [93]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(1984.9744, grad_fn=<DivBackward0>)


In [94]:
# Compute gradients
loss.backward()

# parameters update
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

# Let's verify that the loss is actually lower
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(1434.4601, grad_fn=<DivBackward0>)


In [95]:
# re-initialize the gradients to zero
w.grad.zero_()
b.grad.zero_()

# repeat the parameter update based on gradients
loss.backward()
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

preds = model(inputs)
loss = mse(preds, targets)
print(loss)


tensor(1062.3262, grad_fn=<DivBackward0>)


In [96]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(94.7028, grad_fn=<DivBackward0>)


In [97]:
print(preds)
print(targets)

tensor([[ 53.4854,  71.1355],
        [ 79.4850,  97.0326],
        [130.8038, 139.9011],
        [ -1.7689,  41.9343],
        [110.8119, 109.8299]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [98]:
# linear regression using PyTorch built-ins

import torch.nn as nn

# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [100]:
from torch.utils.data import TensorDataset

# Define dataset
train_ds = TensorDataset(inputs, targets)

from torch.utils.data import DataLoader

# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [101]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[103.,  43.,  36.],
        [ 91.,  87.,  65.],
        [101.,  44.,  37.],
        [102.,  43.,  37.],
        [ 87., 135.,  57.]])
tensor([[ 20.,  38.],
        [ 80., 102.],
        [ 21.,  38.],
        [ 22.,  37.],
        [118., 134.]])


In [102]:
# Define model
model = nn.Linear(3, 2)

# Parameters
list(model.parameters())

# Import nn.functional
import torch.nn.functional as F
loss_fn = F.mse_loss

loss = loss_fn(model(inputs), targets)
print(loss)

# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

tensor(13138.7725, grad_fn=<MseLossBackward0>)


In [103]:
# Training the model

# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [104]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 537.2679
Epoch [20/100], Loss: 147.8839
Epoch [30/100], Loss: 128.2723
Epoch [40/100], Loss: 89.1374
Epoch [50/100], Loss: 49.0374
Epoch [60/100], Loss: 21.5867
Epoch [70/100], Loss: 44.5280
Epoch [80/100], Loss: 26.6871
Epoch [90/100], Loss: 32.7600
Epoch [100/100], Loss: 18.7048


In [106]:
print(model(inputs))
print(targets)

tensor([[ 57.8430,  71.2984],
        [ 82.7410,  97.7221],
        [114.7756, 136.7427],
        [ 25.8661,  43.1475],
        [100.2645, 110.5900],
        [ 56.7555,  70.2703],
        [ 82.6872,  97.2947],
        [115.1543, 137.0840],
        [ 26.9537,  44.1757],
        [101.2983, 111.1907],
        [ 57.7893,  70.8710],
        [ 81.6534,  96.6940],
        [114.8293, 137.1701],
        [ 24.8323,  42.5468],
        [101.3521, 111.6181]], grad_fn=<AddmmBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])


In [108]:
# Prediction for a specific case
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.8541, 67.9163]], grad_fn=<AddmmBackward0>)

In [109]:
# NN modeling

model2 = nn.Sequential(
    nn.Linear(3, 4), # hidden layer
    nn.Sigmoid(),    # non-linearity, activation function
    nn.Linear(4, 2)  # output layer
)

opt = torch.optim.SGD(model2.parameters(), lr=1e-3)

fit(100, model2, F.mse_loss, opt, train_dl)

Epoch [10/100], Loss: 9820.2803
Epoch [20/100], Loss: 3121.2239
Epoch [30/100], Loss: 2126.2554
Epoch [40/100], Loss: 2384.7454
Epoch [50/100], Loss: 2420.4482
Epoch [60/100], Loss: 4236.0117
Epoch [70/100], Loss: 3013.8711
Epoch [80/100], Loss: 1944.3519
Epoch [90/100], Loss: 1295.6188
Epoch [100/100], Loss: 2064.5112
